In [4]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

from tensorflow import keras
from keras.models import Sequential
from keras import Input
from keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.callbacks import EarlyStopping

from lightgbm import LGBMRegressor
import random
import xgboost as xgb

In [2]:
curr_dir = os.getcwd()
curr_dir

'C:\\weather'

In [3]:
forders_tr = os.listdir(curr_dir+"\\uv_0719")
forders_tr

['202001_uv.csv',
 '202002_uv.csv',
 '202003_uv.csv',
 '202004_uv.csv',
 '202005_uv.csv',
 '202006_uv.csv',
 '202007_uv.csv',
 '202008_uv.csv',
 '202009_uv.csv',
 '202010_uv.csv',
 '202011_uv.csv',
 '202012_uv.csv',
 '202101_uv.csv',
 '202102_uv.csv',
 '202103_uv.csv',
 '202104_uv.csv',
 '202105_uv.csv',
 '202106_uv.csv',
 '202107_uv.csv',
 '202108_uv.csv',
 '202109_uv.csv',
 '202110_uv.csv',
 '202111_uv.csv',
 '202112_uv.csv']

In [4]:
forders_tr1 = []
for i in range(0,9):
    df = pd.read_csv(os.path.join(curr_dir+"\\\\uv_0719\\")+forders_tr[i])
    d = list(df.columns)
    for j in range(1,len(d)):
        d[j] = d[j].replace("20200"+str(i+1)+"_uv.", "")
    df.columns = d
    df.drop('Unnamed: 0', axis=1, inplace=True)
    forders_tr1.append(df)

for i in range(9,12):
    df = pd.read_csv(os.path.join(curr_dir+"\\uv_0719\\")+forders_tr[i])
    d = list(df.columns)
    for j in range(1,len(d)):
        d[j] = d[j].replace("2020"+str(i+1)+"_uv.", "")
    df.columns = d
    df.drop('Unnamed: 0', axis=1, inplace=True)
    forders_tr1.append(df)

    
for i in range(12,21):
    df = pd.read_csv(os.path.join(curr_dir+"\\uv_0719\\")+forders_tr[i])
    d = list(df.columns)
    for j in range(1,len(d)):
        d[j] = d[j].replace("20210"+str(i+1-12)+"_uv.", "")
    df.columns = d
    df.drop('Unnamed: 0', axis=1, inplace=True)
    forders_tr1.append(df)

for i in range(21,24):
    df = pd.read_csv(os.path.join(curr_dir+"\\uv_0719\\")+forders_tr[i])
    d = list(df.columns)
    for j in range(1,len(d)):
        d[j] = d[j].replace("2021"+str(i+1-12)+"_uv.", "")
    df.columns = d
    df.drop('Unnamed: 0', axis=1, inplace=True)
    forders_tr1.append(df)

In [6]:
# 새로운 데이터 프레임 생성
train = pd.DataFrame()

for files in forders_tr1:
    train = pd.concat([train, files])
train

,yyyymmdd,hhnn,stn,lon,lat,uv,band1,band2,band3,band4,...,band12,band13,band14,band15,band16,solarza,sateza,esr,height,landtype
0,20200101,0,13,126.16000,33.30000,0.0,0.00000,0.00000,0.00000,-0.00075,...,250.75924,269.97608,269.70811,268.61702,259.10334,165.17963,38.96359,-9.50910,71.00,0
1,20200101,0,105,128.89098,37.75147,0.0,-0.00057,0.00057,0.00030,-0.00030,...,246.67115,268.22810,268.41686,268.01723,256.99670,163.20549,43.95463,-9.40930,26.04,3
2,20200101,0,108,126.96500,37.57100,-999.0,0.00000,0.00000,-0.00089,-0.00075,...,241.24994,258.33982,258.18692,257.58363,250.50021,162.48279,43.77999,-9.37761,85.50,3
3,20200101,0,112,126.62490,37.47772,0.0,-0.00057,-0.00057,-0.00060,0.00015,...,241.55488,258.54927,258.64012,258.03883,251.00820,162.39995,43.67046,-9.37422,68.99,3
4,20200101,0,115,130.89864,37.48129,0.0,0.00000,-0.00057,-0.00030,0.00075,...,244.56708,264.88004,263.95472,262.33534,253.25397,164.19725,43.74521,-9.45200,222.80,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66955,20211231,2350,146,127.11900,35.84080,0.1,-0.00057,0.00000,-0.00060,0.00000,...,244.71129,264.82744,264.95943,264.36433,254.82929,162.52921,41.81018,-8.87355,62.90,3
66956,20211231,2350,152,129.33472,35.58250,0.0,0.00000,0.00000,-0.00179,-0.00015,...,241.75728,259.19303,257.86443,257.39333,249.08495,163.96606,41.52495,-8.99640,82.00,2
66957,20211231,2350,156,126.89158,35.17294,0.0,0.00000,0.00000,0.00000,0.00105,...,242.90744,260.33453,260.00405,259.18646,250.92709,162.84572,41.04849,-8.87987,72.38,3
66958,20211231,2350,159,129.03203,35.10468,0.0,-0.00057,-0.00057,0.00089,-0.00015,...,242.68420,259.36253,258.24370,256.93511,247.71243,164.15158,40.96396,-8.99480,69.56,3


In [13]:
t = train[train['hhnn']>=600]
t1 = t[t['hhnn']<=1950]
t1
t1 = t1.reset_index()
t1 = t1.drop('index',axis=1)
t1

,yyyymmdd,hhnn,stn,lon,lat,uv,band1,band2,band3,band4,...,band12,band13,band14,band15,band16,solarza,sateza,esr,height,landtype
0,20200101,600,13,126.16000,33.30000,0.0,-0.00057,0.00000,-0.00119,0.00045,...,248.77160,267.05904,267.26159,266.49434,257.96958,110.09355,38.96359,-3.54492,71.00,0
1,20200101,600,105,128.89098,37.75147,0.0,-0.00057,-0.00057,-0.00030,-0.00015,...,247.04170,269.99255,270.23960,269.78995,257.77950,109.07770,43.95463,-3.37391,26.04,3
2,20200101,600,108,126.96500,37.57100,-999.0,-0.00057,-0.00057,0.00000,-0.00045,...,241.50417,258.90969,259.24068,258.92424,251.51370,110.52159,43.77999,-3.60577,85.50,3
3,20200101,600,112,126.62490,37.47772,0.0,0.00000,-0.00057,0.00089,0.00015,...,241.63086,258.92862,259.27807,258.90548,251.69508,110.76517,43.67046,-3.64481,68.99,3
4,20200101,600,115,130.89864,37.48129,0.0,-0.00057,-0.00057,0.00030,-0.00045,...,248.23059,270.81214,270.04064,269.11152,256.78573,107.47453,43.74521,-3.11476,222.80,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
921055,20211231,1950,146,127.11900,35.84080,0.1,0.00000,0.00000,0.00000,-0.00030,...,246.27493,267.70492,267.90895,267.53500,257.07331,116.03756,41.81018,-2.88108,62.90,3
921056,20211231,1950,152,129.33472,35.58250,0.0,0.00000,0.00057,-0.00060,-0.00015,...,246.78719,268.79813,269.12337,268.78779,257.70337,117.77223,41.52495,-3.15269,82.00,2
921057,20211231,1950,156,126.89158,35.17294,0.0,0.00000,-0.00057,0.00060,-0.00090,...,247.57042,268.63088,268.87165,268.30897,257.74144,115.72208,41.04849,-2.81930,72.38,3
921058,20211231,1950,159,129.03203,35.10468,0.0,0.00000,0.00000,-0.00149,-0.00030,...,252.05012,277.43607,277.42225,276.64580,261.63172,117.43586,40.96396,-3.09132,69.56,3


In [ ]:
# 27357 => -999 개수

In [62]:
# pd.set_option('display.max_columns', None)
# pd.options.display.max_columns = 20
pd.set_option('display.max_columns', 30)
# pd.set_option('display.max_rows', N3ne)
a1 = t1[t1['uv']<=5]
a2 = a1[a1['uv']>=0]
a = a2['uv'].value_counts()
a.sort_index().sum()

791486

In [63]:
b1 = t1[t1['uv']>5]
b2 = b1[b1['uv']<=10]
b = b2['uv'].value_counts()
b.sort_index().sum()

98408

In [64]:
c1 = t1[t1['uv']>10]
c = c1['uv'].value_counts()
c.sort_index().sum()

3809

In [65]:
aa = t1['uv'].value_counts()
aa.sum()

921060

In [61]:
aa

0.0     177286
0.1      65274
0.2      41323
0.3      32239
0.4      31084
         ...  
14.4         1
18.9         1
15.8         1
18.5         1
23.6         1
Name: uv, Length: 141, dtype: int64